# IMPORT DES LIBRAIRIES

In [9]:
import pandas as pd
import numpy as np

# OPTIONS PANDAS 

In [10]:
pd.options.display.float_format = '{:.0f}'.format

# OUVERTURE DE NOTRE DATASET

In [11]:
df = pd.read_csv('../DATA/IN/extract_gold_dvf.csv', sep=';')
df.head()

C:\Users\jonat\AppData\Local\Temp\ipykernel_30704\249013501.py:1: DtypeWarning: Columns (15,17,24) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv('../DATA/IN/extract_gold_dvf.csv', sep=';')


,NoDisp,DateMutation,ValeurFonciere,NoVoie,BTQ,TypeVoie,CodeVoie,Voie,CP,Commune,...,SurfaceCarrez4emeLot,CinquiemeLot,SurfaceCarrez5emeLot,NombreLots,CodeTypeLocal,SurfaceReelleBati,NombrePiecesPrincipales,NatureCulture,NatureCultureSpeciale,SurfaceTerrain
0,1,2021-04-26,311650,16,NaN,RUE,0270,DE LA GRANGE DIMIERE,33850,LEOGNAN,...,NaN,NaN,NaN,1,1,84,4,NaN,NaN,NaN
1,1,2021-04-26,122000,28,NaN,CRS,0840,DE VERDUN,33470,GUJAN MESTRAS,...,NaN,NaN,NaN,1,1,34,2,NaN,NaN,NaN
2,1,2021-04-23,420612,47,NaN,RUE,0144,JEAN MARIE PELT,33380,MIOS,...,NaN,NaN,NaN,0,1,130,4,S,NaN,607
3,1,2021-04-19,167000,1,NaN,RUE,0030,ARMAND DANEY,33470,GUJAN MESTRAS,...,NaN,NaN,NaN,2,2,49,2,NaN,NaN,NaN
4,1,2021-05-05,450000,11,NaN,ALL,1180,FRERES DUPUY CAZAUX,33260,LA TESTE-DE-BUCH,...,NaN,NaN,NaN,0,1,105,4,S,NaN,684


In [12]:
# df = pd.read_csv('../DATA/IN/extract_gold_dvf.csv', sep=';')
# 
# # Ajout d'une colonne ID qui sera un identifiant unique pour chaque ligne (commençant à 1)
# df['ID'] = range(1, len(df) + 1)
# 
# # Vérification de l'ajout de la colonne ID
# df[['ID'] + df.columns.tolist()[:-1]].head()

# # Modification of ValeurFonciere
# df['ValeurFonciere'] = df['ValeurFonciere'].str.replace(',', '.').astype(float)
# 
# # Modification of SurfaceCarrez
# df['SurfaceCarrez1erLot'] = df['SurfaceCarrez1erLot'].str.replace(',', '.').astype(float)
# df['SurfaceCarrez2emeLot'] = df['SurfaceCarrez2emeLot'].str.replace(',', '.').astype(float)
# df['SurfaceCarrez3emeLot'] = df['SurfaceCarrez3emeLot'].str.replace(',', '.').astype(float)
# df['SurfaceCarrez4emeLot'] = df['SurfaceCarrez4emeLot'].str.replace(',', '.').astype(float)
# df['SurfaceCarrez5emeLot'] = df['SurfaceCarrez5emeLot'].str.replace(',', '.').astype(float)

# Modification of NoVoie and SurfaceTerrain and& SurfaceCarrez
# 
# #Transform NA values to 0
# df['NoVoie'] = df['NoVoie'].fillna(0)
# df['SurfaceTerrain'] = df['SurfaceTerrain'].fillna(0)
# 
# df['SurfaceCarrez1erLot'] = df['SurfaceCarrez1erLot'].fillna(0)
# df['SurfaceCarrez2emeLot'] = df['SurfaceCarrez2emeLot'].fillna(0)
# df['SurfaceCarrez3emeLot'] = df['SurfaceCarrez3emeLot'].fillna(0)
# df['SurfaceCarrez4emeLot'] = df['SurfaceCarrez4emeLot'].fillna(0)
# df['SurfaceCarrez5emeLot'] = df['SurfaceCarrez5emeLot'].fillna(0)

# # Modification of NoVoie type
# df['NoVoie'] = df['NoVoie'].astype(int)

                                                # Modification of non numeric values of NombrePiecesPrincipales 
                                                df['NombrePiecesPrincipales'] = (
                                                    pd.to_numeric(df['NombrePiecesPrincipales'],
                                                                  errors='coerce')
                                                    .fillna(0)
                                                )

# Modification of NombrePiecesPrincipales type
# # df['NombrePiecesPrincipales'] = df['NombrePiecesPrincipales'].astype(int)
# 
# # Convertir la colonne 'DateMutation' en datetime si ce n'est pas déjà fait
# df['DateMutation'] = pd.to_datetime(df['DateMutation'])
# 
# # Extraire l'année 
# df['Year'] = df['DateMutation'].dt.year

# Calcul de la surface carrez totale
# df['SurfaceCarrez'] = df['SurfaceCarrez1erLot'] + df['SurfaceCarrez2emeLot'] + df['SurfaceCarrez3emeLot'] + df['SurfaceCarrez4emeLot'] + df['SurfaceCarrez5emeLot']

################################################################################OUTLIERS################################################################################
# ISOLER LES BIENS AVEC TERRAINS
bien_avec_terrain = df.loc[(df['SurfaceTerrain'] > 0)]

df['PrixTerrainM2'] = df['ValeurFonciere'] / df['SurfaceTerrain']
bien_avec_terrain['PrixTerrainM2'] = bien_avec_terrain['ValeurFonciere'] / bien_avec_terrain['SurfaceTerrain']
# Remplacez les valeurs infinies par zéro.
df['PrixTerrainM2'].replace(np.inf, 0, inplace=True)
bien_avec_terrain['PrixTerrainM2'].replace(np.inf, 0, inplace=True)

# CALCUL DE NOTRE PREMIER QUARTILE
q1_prix_terrain = bien_avec_terrain['PrixTerrainM2'].quantile([.25])
q1_prix_terrain = q1_prix_terrain[0.25]
q3_prix_terrain = bien_avec_terrain['PrixTerrainM2'].quantile([.75])
q3_prix_terrain = q3_prix_terrain[0.75]

# CALCUL DE NOTRE ECART INTER QUARTILES
iqr_prix_terrain = q3_prix_terrain - q1_prix_terrain



outliers_prix_terrain_m2_inferieur = q1_prix_terrain - (1.5 * iqr_prix_terrain)
outliers_prix_terrain_m2_superieur = q3_prix_terrain + (1.5 * iqr_prix_terrain)



outliers_surface_terrain = df.loc[(df['PrixTerrainM2'] > outliers_prix_terrain_m2_superieur) | (df['PrixTerrainM2'] < outliers_prix_terrain_m2_inferieur)]

df['PrixBatiM2'] = df['ValeurFonciere'] / df['SurfaceReelleBati']

# CALCUL DE NOTRE PREMIER QUARTILE
q1_prix_bati = df['PrixBatiM2'].quantile([.25])
q1_prix_bati = q1_prix_bati[0.25]
q3_prix_bati = df['PrixBatiM2'].quantile([.75])
q3_prix_bati = q3_prix_bati[0.75]

# CALCUL DE NOTRE ECART INTER QUARTILES
iqr_prix_bati = q3_prix_bati - q1_prix_bati

outliers_prix_bati_m2_inferieur = q1_prix_bati - (1.5 * iqr_prix_bati)
outliers_prix_bati_m2_superieur = q3_prix_bati + (1.5 * iqr_prix_bati)

outliers_surface_bati = df.loc[(df['PrixBatiM2'] > outliers_prix_bati_m2_superieur) | (df['PrixBatiM2'] < outliers_prix_bati_m2_inferieur)]

# ISOLER LES BIENS AVEC UNE SURFACE CARREZ
bien_avec_carrez = df.loc[(df['SurfaceCarrez'] > 0)]

df['PrixCarrezM2'] = df['ValeurFonciere'] / df['SurfaceCarrez']
bien_avec_carrez['PrixCarrezM2'] = bien_avec_carrez['ValeurFonciere'] / bien_avec_carrez['SurfaceCarrez']
# Remplacez les valeurs infinies par zéro.
df['PrixCarrezM2'].replace(np.inf, 0, inplace=True)
bien_avec_carrez['PrixCarrezM2'].replace(np.inf, 0, inplace=True)

# CALCUL DE NOTRE PREMIER QUARTILE
q1_prix_carrez = bien_avec_carrez['PrixCarrezM2'].quantile([.25])
q1_prix_carrez = q1_prix_carrez[0.25]
q3_prix_carrez = bien_avec_carrez['PrixCarrezM2'].quantile([.75])
q3_prix_carrez = q3_prix_carrez[0.75]

# CALCUL DE NOTRE ECART INTER QUARTILES
iqr_prix_carrez = q3_prix_carrez - q1_prix_carrez

outliers_prix_carrez_m2_inferieur = q1_prix_carrez - (1.5 * iqr_prix_carrez)
outliers_prix_carrez_m2_superieur = q3_prix_carrez + (1.5 * iqr_prix_carrez)

outliers_surface_carrez = df.loc[(df['PrixCarrezM2'] > outliers_prix_carrez_m2_superieur) | (df['PrixCarrezM2'] < outliers_prix_carrez_m2_inferieur)]

# Concaténation des ID des outliers
ids_outliers = pd.concat([outliers_surface_bati['ID'], outliers_surface_terrain['ID'], outliers_surface_carrez['ID']])

# Suppression des duplicatas
ids_outliers = ids_outliers.drop_duplicates()

# Filtrage de df pour exclure les outliers
df_clean = df[~df['ID'].isin(ids_outliers)]

# CALCUL DE NOTRE PREMIER QUARTILE
q1_nb_piece = df['NombrePiecesPrincipales'].quantile([.25])
q1_nb_piece = q1_nb_piece[0.25]
q3_nb_piece = df['NombrePiecesPrincipales'].quantile([.75])
q3_nb_piece = q3_nb_piece[0.75]

# CALCUL DE NOTRE ECART INTER QUARTILES
iqr_nb_piece = q3_nb_piece - q1_nb_piece

outliers_nb_pieces_inferieur = q1_nb_piece - (1.5 * iqr_nb_piece)
outliers_nb_pieces_superieur = q3_nb_piece + (1.5 * iqr_nb_piece)

outliers_nb_pieces = df.loc[(df['NombrePiecesPrincipales'] > outliers_nb_pieces_superieur) | (df['NombrePiecesPrincipales'] < outliers_nb_pieces_inferieur)]

# Concaténation des ID des outliers
ids_outliers = pd.concat([outliers_surface_bati['ID'], outliers_surface_terrain['ID'], outliers_surface_carrez['ID'], outliers_nb_pieces['ID']])

# Suppression des duplicatas
ids_outliers = ids_outliers.drop_duplicates()

# Filtrage de df pour exclure les outliers
df_clean_with_nb_piece = df[~df['ID'].isin(ids_outliers)]

################################################################################FIN OUTLIERS################################################################################

# Préparation de df_model en excluant les colonnes spécifiées dans votre code initial
columns_to_drop = ['NoDisp', 'NoVoie', 'BTQ', 'TypeVoie', 'CodeVoie', 'Voie', 'Commune',
                   'CodeDepartement', 'CodeCommune', 'PrefixeSection', 'Section', 'NoPlan',
                   'PremierLot', 'SurfaceCarrez1erLot', 'DeuxiemeLot', 'SurfaceCarrez2emeLot',
                   'TroisiemeLot', 'SurfaceCarrez3emeLot', 'QuatriemeLot', 'SurfaceCarrez4emeLot',
                   'CinquiemeLot', 'SurfaceCarrez5emeLot', 'NatureCulture', 'NatureCultureSpeciale',
                   'DateMutation', 'ID', 'PrixTerrainM2', 'PrixBatiM2', 'PrixCarrezM2']
0,
# Créer df_model
df_model = df_clean_with_nb_piece.drop(columns=columns_to_drop)

df_model = df_model.dropna(subset=['ValeurFonciere', 'SurfaceReelleBati', 'CP'])

# Réinitialiser l'index pour s'assurer que l'alignement est conservé après la suppression des NaN
df_model.reset_index(drop=True, inplace=True)

C:\Users\jonat\AppData\Local\Temp\ipykernel_30704\3730214780.py:1: DtypeWarning: Columns (15,17,24) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv('../DATA/IN/extract_gold_dvf.csv', sep=';')
C:\Users\jonat\AppData\Local\Temp\ipykernel_30704\3730214780.py:58: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  bien_avec_terrain['PrixTerrainM2'] = bien_avec_terrain['ValeurFonciere'] / bien_avec_terrain['SurfaceTerrain']
C:\Users\jonat\AppData\Local\Temp\ipykernel_30704\3730214780.py:61: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-

In [13]:
df_model

,ValeurFonciere,CP,NombreLots,CodeTypeLocal,SurfaceReelleBati,NombrePiecesPrincipales,SurfaceTerrain,Year,SurfaceCarrez
0,311650,33850,1,1,84,4,0,2021,84
1,122000,33470,1,1,34,2,0,2021,0
2,420612,33380,0,1,130,4,607,2021,0
3,167000,33470,2,2,49,2,0,2021,49
4,450000,33260,0,1,105,4,684,2021,0
...,...,...,...,...,...,...,...,...,...
147504,234150,33990,0,1,103,6,880,2020,0
147505,159300,33340,0,1,103,5,162,2020,0
147506,159300,33340,0,1,103,5,500,2020,0
147507,283925,33780,0,1,132,6,601,2020,0
